In [1]:
import pandas as pd
import pyarrow.parquet as pq

## Creat a static dictionary for useful columns

In [2]:
# static cols
dic = {
    'windmills':['GSRN','Turbine_type','Parent_GSRN','BBR_municipal','Placement','UTM_x','UTM_y','Capacity_kw','Rotor_diameter','Navhub_height']
}

In [3]:
windmills = pq.read_table('data/ITU_DATA/masterdatawind.parquet').to_pandas()[dic['windmills']]

In [4]:
windmills.drop_duplicates(subset ="GSRN", keep = 'first', ignore_index = True, inplace = True)

In [6]:
windmills[windmills['Turbine_type'] == 'M'].head(20)

,GSRN,Turbine_type,Parent_GSRN,BBR_municipal,Placement,UTM_x,UTM_y,Capacity_kw,Rotor_diameter,Navhub_height
1,570714700000000027,M,570714700000050459,101,LAND,720898.353,6171174.877,225.0,27.0,30.0
2,570714700000000034,M,570714700000050459,101,LAND,720993.352,6171226.877,225.0,27.0,30.0
3,570714700000000041,M,570714700000050466,101,LAND,727504.304,6178385.864,600.0,44.0,50.0
4,570714700000000058,M,570714700000050466,101,LAND,727628.303,6178385.865,600.0,44.0,50.0
5,570714700000000065,M,570714700000050466,101,LAND,727817.302,6178365.866,600.0,44.0,50.0
6,570714700000000072,M,570714700000050053,167,LAND,717288.000,6167432.000,300.0,31.0,30.0
7,570714700000000089,M,570714700000050053,167,LAND,717423.000,6167422.000,300.0,31.0,30.0
8,570714700000000096,M,570714700000050053,167,LAND,717556.000,6167412.000,300.0,31.0,30.0
10,570714700000000119,M,570714700000050060,167,LAND,718603.000,6167242.000,300.0,31.0,30.0
11,570714700000000126,M,570714700000050060,167,LAND,718736.000,6167233.000,300.0,31.0,30.0


In [7]:
windmills.shape

(9853, 10)

In [12]:
 windmills['GSRN'] = [x+'z' for x in windmills['GSRN']]

In [23]:
windmills['Parent_GSRN'].fillna('',inplace = True)

In [24]:
windmills['Parent_GSRN'] = [x+'z' for x in windmills['Parent_GSRN']]

In [26]:
windmills.to_csv('data/windmills_gis.csv', index=False)

In [32]:
aggs = {'UTM_x':lambda x : x.mean(),
        'UTM_y':lambda x : x.mean()}

In [48]:
f_temp = windmills[windmills['Turbine_type'] == 'M'].groupby('Parent_GSRN', as_index = False).agg(aggs).rename(columns={'Parent_GSRN':'GSRN','UTM_x':'x_t','UTM_y':'y_t'})

In [49]:
f_temp

,GSRN,x_t,y_t
0,570714700000005640z,642373.900000,6.139972e+06
1,570714700000012037z,664423.733000,6.072868e+06
2,570714700000012051z,664630.731000,6.072840e+06
3,570714700000012105z,728122.163500,6.165601e+06
4,570714700000050008z,684100.400000,6.097951e+06
...,...,...,...
574,570715000000258107z,636075.348135,6.275118e+06
575,570715000001613493z,415507.469388,6.172906e+06
576,570715000001761682z,464444.000000,6.283758e+06
577,571313134700404281z,453783.500000,6.281010e+06


In [61]:
f = pd.merge(windmills, f_temp, on='GSRN',how='left')

In [76]:
f.shape

(9853, 10)

In [69]:
f[['UTM_x']] = f.apply(lambda x: x[['UTM_x']].fillna(value = x['x_t']), axis=1)[['UTM_x']]

In [71]:
f[['UTM_y']] = f.apply(lambda x: x[['UTM_y']].fillna(value = x['y_t']), axis=1)[['UTM_y']]

In [75]:
f.drop(columns=['x_t','y_t'], inplace=True)

In [104]:
f[f['Parent_GSRN'] != 'z'].groupby(['Parent_GSRN','Navhub_height'],as_index = False).count()

,Parent_GSRN,Navhub_height,GSRN,Turbine_type,BBR_municipal,Placement,UTM_x,UTM_y,Capacity_kw,Rotor_diameter
0,570714700000005640z,30.0,1,1,1,1,1,1,1,1
1,570714700000012037z,45.0,1,1,1,1,1,1,1,1
2,570714700000012051z,45.0,1,1,1,1,1,1,1,1
3,570714700000012105z,24.0,6,6,6,6,6,6,6,6
4,570714700000050008z,45.0,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...
646,570715000000091803z,20.0,2,2,2,2,2,2,2,2
647,570715000000258107z,81.6,111,111,111,111,111,111,111,111
648,570715000001613493z,105.0,49,49,49,49,49,49,49,49
649,570715000001761682z,18.5,2,2,2,2,2,2,2,2


In [107]:
f.to_csv('data/windmills_gis.csv', index=False)

In [158]:
pd.set_option('display.float_format',lambda x : '%.3f' % x)

In [195]:
windmills_multi = pd.read_excel('data/Windmills_to_Multi-Grids.xlsx')

In [196]:
windmills_multi['Parent_GSRN'].fillna(value=0, inplace = True)
windmills_multi['grid'].fillna(value=0, inplace = True)

In [197]:
windmills_multi

,GSRN,Turbine_type,Parent_GSRN,BBR_municipal,Placement,UTM_x,UTM_y,Capacity_kw,Rotor_diameter,Navhub_height,BUFF_DIST,grid
0,570714700000000027,M,570714700000050496.000,101,LAND,720898.353,6171174.877,225.000,27.000,30.000,30000,609.000
1,570714700000000027,M,570714700000050496.000,101,LAND,720898.353,6171174.877,225.000,27.000,30.000,30000,610.000
2,570714700000000027,M,570714700000050496.000,101,LAND,720898.353,6171174.877,225.000,27.000,30.000,30000,611.000
3,570714700000000027,M,570714700000050496.000,101,LAND,720898.353,6171174.877,225.000,27.000,30.000,30000,651.000
4,570714700000000027,M,570714700000050496.000,101,LAND,720898.353,6171174.877,225.000,27.000,30.000,30000,652.000
...,...,...,...,...,...,...,...,...,...,...,...,...
76172,571313179100196008,W,0.000,265,LAND,694663.000,6175328.000,1200.000,29.000,59.500,30000,736.000
76173,571313179100196008,W,0.000,265,LAND,694663.000,6175328.000,1200.000,29.000,59.500,30000,737.000
76174,571313179100196008,W,0.000,265,LAND,694663.000,6175328.000,1200.000,29.000,59.500,30000,738.000
76175,571313179100196008,W,0.000,265,LAND,694663.000,6175328.000,1200.000,29.000,59.500,30000,778.000


In [198]:
windmills_multi['Parent_GSRN'] = windmills_multi['Parent_GSRN'].astype(int)
windmills_multi['grid'] = windmills_multi['grid'].astype(int)
windmills_multi['grid'] = windmills_multi['grid'].astype(str)

In [199]:
def ab(df):
    return','.join(df.values)
    
windmills_multi_merged = windmills_multi.groupby('GSRN')['grid'].apply(ab).reset_index()


In [200]:
windmills_multi_merged

,GSRN,grid
0,570714700000000027,"609,610,611,651,652,653,654,694,695,696"
1,570714700000000034,"609,610,611,651,652,653,654,694,695,696"
2,570714700000000041,"610,611,652,653,654,694,695,696"
3,570714700000000058,"610,611,652,653,654,694,695,696"
4,570714700000000065,"610,611,652,653,654,694,695,696"
...,...,...
7515,571313174001699894,"728,729,730,731,770,771,772,773,813,814"
7516,571313174001699900,"728,729,730,731,770,771,772,773,813,814"
7517,571313174001764448,"607,608,609,649,650,651,691,692,693,734"
7518,571313174116253585,"652,653,693,694,695,696,735,736,737,738,778,779"


In [201]:
windmills_grid= pd.read_excel('data/Point_SpatialJoin.xlsx')

In [202]:
windmills_grid['grid'].fillna(value=0, inplace = True)
windmills_grid['grid'] = windmills_grid['grid'].astype(int)
windmills_grid['grid'] = windmills_grid['grid'].astype(str)

In [206]:
windmills_multi_merged

,GSRN,grid_in_range
0,570714700000000027,"609,610,611,651,652,653,654,694,695,696"
1,570714700000000034,"609,610,611,651,652,653,654,694,695,696"
2,570714700000000041,"610,611,652,653,654,694,695,696"
3,570714700000000058,"610,611,652,653,654,694,695,696"
4,570714700000000065,"610,611,652,653,654,694,695,696"
...,...,...
7515,571313174001699894,"728,729,730,731,770,771,772,773,813,814"
7516,571313174001699900,"728,729,730,731,770,771,772,773,813,814"
7517,571313174001764448,"607,608,609,649,650,651,691,692,693,734"
7518,571313174116253585,"652,653,693,694,695,696,735,736,737,738,778,779"


In [204]:
windmills_multi_merged.rename(columns={'grid':'grid_in_range'}, inplace=True)

In [207]:
f = pd.merge(windmills_grid, windmills_multi_merged, on='GSRN',how='left')

In [208]:
f

,GSRN,Turbine_type,Parent_GSRN,BBR_municipal,Placement,UTM_x,UTM_y,Capacity_kw,Rotor_diameter,Navhub_height,grid,grid_in_range
0,0,H,nan,760,NaN,nan,nan,11.000,13.000,18.200,0,NaN
1,570714700000000027,M,570714700000050496.000,101,LAND,720898.353,6171174.877,225.000,27.000,30.000,653,"609,610,611,651,652,653,654,694,695,696"
2,570714700000000034,M,570714700000050496.000,101,LAND,720993.352,6171226.877,225.000,27.000,30.000,653,"609,610,611,651,652,653,654,694,695,696"
3,570714700000000041,M,570714700000050496.000,101,LAND,727504.304,6178385.864,600.000,44.000,50.000,653,"610,611,652,653,654,694,695,696"
4,570714700000000058,M,570714700000050496.000,101,LAND,727628.303,6178385.865,600.000,44.000,50.000,653,"610,611,652,653,654,694,695,696"
...,...,...,...,...,...,...,...,...,...,...,...,...
9848,571313179100075143,H,nan,219,NaN,nan,nan,10.000,13.000,18.200,0,NaN
9849,571313179100118352,H,nan,260,NaN,nan,nan,10.000,7.000,18.000,0,NaN
9850,571313179100196008,W,nan,265,LAND,694663.000,6175328.000,1200.000,29.000,59.500,736,"652,653,693,694,695,696,735,736,737,738,778,779"
9851,571313179100992129,H,nan,250,NaN,nan,nan,25.000,11.000,23.500,0,NaN


In [209]:
f.to_csv('data/windmill_cleaned.csv')

In [5]:
weather_grids = pd.read_csv('data/ITU_DATA/prognosis/grid_coordinates.csv') 
print(len(weather_grids))
weather_grids.head(10)

354


,grid,utm_x,utm_y
0,190,895390,6112543
1,191,899875,6128134
2,231,874801,6101548
3,232,879280,6117148
4,233,883767,6132746
5,273,858691,6106142
6,274,863172,6121751
7,275,867660,6137358
8,314,838109,6095118
9,315,842583,6110737


In [6]:
observed_pressure = pq.read_table('data/ITU_DATA/observations/observed_pressure.parquet').to_pandas()

In [7]:
print(len(observed_pressure))
observed_pressure.tail(10)

22890252


,municipal,subtype,value,unit
timestamp_utc,,,,
2019-06-24 10:00:00,Horsens,Time atmosfæretryk,1027.078491,hPa
2019-06-24 07:00:00,Københavns,Time atmosfæretryk,1027.699951,hPa
2019-06-24 10:00:00,Mariagerfjord,Time atmosfæretryk,1027.682251,hPa
2019-06-24 00:00:00,Morsø,Time atmosfæretryk,1026.642578,hPa
2019-06-23 22:00:00,Næstved,Time atmosfæretryk,1026.166016,hPa
2019-06-24 16:00:00,Odsherred,Time atmosfæretryk,1026.213379,hPa
2019-06-24 21:00:00,Odsherred,Time atmosfæretryk,1025.884277,hPa
2019-06-24 06:00:00,Slagelse,Time atmosfæretryk,1027.182861,hPa
2019-06-24 14:00:00,Sorø,Time atmosfæretryk,1026.566650,hPa


In [7]:
observed_wind_speed_high10 = pq.read_table('data/ITU_DATA/observations/observed_wind_speed_high10.parquet').to_pandas()
print(len(observed_wind_speed_high10))
observed_wind_speed_high10.tail(10)

In [8]:
wind_speed_10m = pq.read_table('data/ITU_DATA/prognosis/ENetNEA/wind_speed_10m.parquet').to_pandas()

In [27]:
print(wind_speed_10m.shape)
wind_speed_10m.tail(60)

(291280, 355)


,190,191,231,232,233,273,274,275,314,315,...,1412,1413,1445,1446,1447,1448,1449,1450,1451,predicted_ahead
2020-01-11 14:00:00,11.5,10.8,11.7,9.1,8.6,11.1,9.2,8.7,10.9,11.4,...,15.6,15.7,16.4,16.1,15.6,15.6,15.6,15.5,15.7,51
2020-01-11 15:00:00,12.3,11.8,12.4,9.9,9.2,12.1,9.8,9.3,11.7,12.2,...,15.6,15.6,16.2,15.9,15.9,15.5,15.8,15.8,15.9,52
2020-01-11 16:00:00,12.9,11.8,13.1,10.7,9.5,12.6,10.1,9.8,11.8,12.4,...,15.3,15.2,16.7,16.3,16.0,15.9,15.8,15.8,15.9,53
2020-01-11 17:00:00,13.5,11.6,13.2,11.0,11.2,13.1,10.3,9.8,12.3,13.0,...,14.8,15.0,16.6,16.5,16.5,16.3,16.2,16.0,15.9,54
2020-01-11 18:00:00,13.6,11.8,13.4,11.5,11.5,13.2,10.5,10.1,12.7,13.3,...,15.3,15.0,16.7,16.7,16.6,16.5,16.0,16.2,16.1,55
2020-01-10 00:00:00,7.9,7.2,9.4,5.6,5.9,8.9,6.5,6.2,8.7,9.0,...,10.2,11.1,8.1,7.1,6.7,6.3,5.1,4.7,3.6,1
2020-01-10 01:00:00,8.2,7.2,8.3,5.3,6.0,7.9,6.1,5.3,7.6,7.5,...,12.2,11.9,6.9,6.3,5.3,4.7,4.4,3.6,9.3,2
2020-01-10 02:00:00,6.7,5.7,7.6,4.7,5.2,7.0,2.8,4.1,6.8,6.4,...,11.9,12.0,5.8,4.5,4.1,2.6,3.1,10.5,11.7,3
2020-01-10 03:00:00,5.8,5.2,6.7,3.5,4.2,6.1,3.4,4.3,6.3,4.3,...,11.8,12.1,3.3,2.6,4.0,9.8,10.0,10.7,11.7,4
2020-01-10 04:00:00,3.6,4.9,6.5,2.0,4.1,6.2,3.5,3.0,6.0,3.1,...,12.1,11.6,4.3,9.0,10.5,11.9,12.1,11.9,12.2,5


In [9]:
settlement_2018 = pq.read_table('data/ITU_DATA/settlement/2018.parquet').to_pandas()

In [17]:
print(len(settlement_2018))
settlement_2018.head(20)

155208851


,GSRN,TS_ID,VAERDI,TIME_CET
0,570715000000062988,1471530,0,2018-07-01 23:15:00
1,570715000000062988,1471530,0,2018-07-01 23:30:00
2,570715000000062988,1471530,0,2018-07-01 23:45:00
3,570715000000062988,1471530,0,2018-07-02 00:00:00
4,570715000000062988,1471530,0,2018-07-02 00:15:00
5,570715000000062988,1471530,0,2018-07-02 00:30:00
6,570715000000062988,1471530,0,2018-07-02 00:45:00
7,570715000000062988,1471530,0,2018-07-02 01:00:00
8,570715000000062988,1471530,0,2018-07-02 01:15:00
9,570715000000062988,1471530,0,2018-07-02 01:30:00


In [21]:
settlement_2018[settlement_2018['GSRN'] == '570714700000004766'].head(50)

,GSRN,TS_ID,VAERDI,TIME_CET
530090,570714700000004766,1573189,0,2018-07-01 00:00:00
530091,570714700000004766,1573189,0,2018-07-01 01:00:00
530092,570714700000004766,1573189,0,2018-07-01 02:00:00
530093,570714700000004766,1573189,0,2018-07-01 03:00:00
530094,570714700000004766,1573189,0,2018-07-01 04:00:00
530095,570714700000004766,1573189,0,2018-07-01 05:00:00
530096,570714700000004766,1573189,0,2018-07-01 06:00:00
530097,570714700000004766,1573189,0,2018-07-01 07:00:00
530098,570714700000004766,1573189,0,2018-07-01 08:00:00
530099,570714700000004766,1573189,0,2018-07-01 09:00:00


In [1]:
test_2018 = settlement_2018.take(10)

NameError: name 'settlement_2018' is not defined